In [25]:
import statsapi
import pprint
import datetime as dt
import pandas as pd
import time

In [26]:
date = dt.datetime.now()
today = f'{date.month:02d}/{date.day}/{date.year}'

print(today)

05/25/2023


In [4]:
# 팀 코드

team_id = {'Baltimore Orioles':110, 'Boston Red Sox':111, 'Newyork Yankees':147, 'Tampa Bay Rays':139,'Toronto Blue Jays':141,
          'Chicago White Sox':145, 'Cleveland Guardians':114, 'Detroit Tigers':116, 'Kansas City Royals':118, 'Minnesota Twins':142,
          'Houston Astros':117,'Los Angeles Angels':108, 'Oakland Athletics':133,'Seattle Mariners':136,'Texas Rangers':140,
          'Atlanta Braves':144,'Miami Marlins':146,'New York Mets':121,'Philadelphia Phillies':143,'Washington Nationals':120,
          'Chicago Cubs':112,'Cincinnati Reds':113,'Milwaukee Brewers':158,'Pittsburgh Pirates':134,'St. Louis Cardinals':138,
          'Arizona Diamondbacks':109,'Los Angeles Dodgers':119,'Colorado Rockies':115,'San Diego Padres':135,'San Francisco Giants':137}

# team

In [5]:
# team 데이터 저장
# for key, value in team_id.items():
#     pd.DataFrame(statsapi.get('team', {'teamId':value})).to_json(f'./team/{key}_{value}.json')

In [6]:
# team entity를 위한 team_list (type : dictionary) 생성 (attribute : id, 팀명, 위치, 구장명)
team_entity = dict()

for key, value in team_id.items():
    team = pd.read_json(f'./data/team/{key}_{value}.json')
    team_entity[value] = [team['teams'][0]['id'], team['teams'][0]['name'], team['teams'][0]['venue']['name'],team['teams'][0]['locationName'],\
                        team['teams'][0]['league']['id'], team['teams'][0]['division']['id']]
    
    

In [7]:
pd.DataFrame(team_entity,index=['team_id','team_name','venue','location','location_id','division_id']).transpose().to_csv('./data/team.csv',index=False)

In [8]:
pd.read_csv('./data/team.csv')

,team_id,team_name,venue,location,location_id,division_id
0,110,Baltimore Orioles,Oriole Park at Camden Yards,Baltimore,103,201
1,111,Boston Red Sox,Fenway Park,Boston,103,201
2,147,New York Yankees,Yankee Stadium,Bronx,103,201
3,139,Tampa Bay Rays,Tropicana Field,St. Petersburg,103,201
4,141,Toronto Blue Jays,Rogers Centre,Toronto,103,201
5,145,Chicago White Sox,Guaranteed Rate Field,Chicago,103,202
6,114,Cleveland Guardians,Progressive Field,Cleveland,103,202
7,116,Detroit Tigers,Comerica Park,Detroit,103,202
8,118,Kansas City Royals,Kauffman Stadium,Kansas City,103,202
9,142,Minnesota Twins,Target Field,Minneapolis,103,202


# league

In [9]:
# 아메리칸 리그(서부, 동부, 중부) / 내셔널 리그(서부, 동부, 중부)
AL_division_id = {'AL_west':200, 'AL_east':201, 'AL_central':202}
NL_division_id = {'NL_west':203, 'NL_east':204, 'NL_central':205}

league = {'AL_west' : (103, 200, 'AL','west'), 'AL_east' : (103,201,'AL','east'),'AL_central' : (103,202,'AL','central'),
          'NL_west' : (104, 203, 'NL', 'west'), 'NL_east' : (104,204,'NL','east'),'NL_central' : (104,205,'NL','central')}

pd.DataFrame(league, index=['league_id', 'division_id', 'league', 'division']).transpose().to_csv('./data/league.csv')

# ranking

In [10]:
# 최종 순위에 접근하기 위한 각 해의 마지막 경기 날짜
last_day = ('09/30/2013','09/28/2014','10/04/2015','10/02/2016','10/01/2017','10/01/2018','09/29/2019','09/27/2020','10/03/2021','10/05/2022', f'{date.month:02d}/{date.day}/{date.year}')

In [11]:
# AL / NL 순위파일 저장
# for i in last_day:
#     year = i[-4:]
#     pd.DataFrame(statsapi.standings_data(leagueId=103,date=i)).to_json(f'./data/ranking/AL_{year}.json')
#     pd.DataFrame(statsapi.standings_data(leagueId=104,date=i)).to_json(f'./data/ranking/NL_{year}.json')
    

In [12]:
# 10년동안의 리그-지구 별 순위
ranking_entity = dict()

for year in range(2013, 2024):
    rank = pd.read_json(f'./data/ranking/AL_{year}.json')

    for key, value in AL_division_id.items():

        for seq in range(5):
            league_info = rank[value]['teams'][seq]
            ranking_entity[f'{year}_{key}_{seq}'] = year, 103, value,league_info['team_id'],league_info['div_rank'], league_info['w'], league_info['l']

    rank = pd.read_json(f'./data/ranking/NL_{year}.json')

    for key, value in NL_division_id.items():
        for seq in range(5):
            league_info = rank[value]['teams'][seq]
            ranking_entity[f'{year}_{key}_{seq}'] = year, 104, value,league_info['team_id'],league_info['div_rank'], league_info['w'], league_info['l']

In [13]:
pd.DataFrame(ranking_entity, index = ['year','team_id','ranking','win','loss'])\
    .transpose().to_csv('./data/ranking.csv', index=False)

ValueError: Length of values (7) does not match length of index (5)

# player

In [ ]:
# 몸무게의 값을 found 에서 kg으로 변환하는 함수

def lb_to_kg(lb):
    return lb * 0.45

In [ ]:
# 키를 inch에서 m로 변환하는 함수 

def ft_to_m(ft):
    return round(float(ft.replace('\' ','.').replace('"','')) * 0.3, 3)

In [ ]:
# for year in range(2013, 2024):
#     pd.DataFrame(statsapi.get('sports_players',{'season':year})).to_json(f'./data/player/player_{year}.json')

In [ ]:
# player entity를 위한 10년간 mlb에 속해있던 player들의 정보 (type : dictionary) 생성 
#                       (attribute : 마지막 등록년, 선수 id, 생일, 국적, 키(m), 몸무게(kg), 소속팀(id), 포지션(code, 이름), 데뷔일, 투, 타)
player = dict()

# 10년동안 mlb에 속해있던 선수 id 정보만을 사용하기 위한 데이터 (type : dictionary)
player_id = dict() 

# 해당 년도의 소속 팀을 저장하기 위한 데이터 (type :  dictionary) -> player affiliation entity를 생성하기 위해서 필요
player_affiliation = dict() 

for year in range(2013, 2024):
    players_record = pd.read_json(f'./data/player/player_{year}.json')

    for i in players_record['people']:
        # 소속은 되어있으나 아직 데뷔하지 못한 인원의 데뷔일은 1000-01-01처리
        if 'mlbDebutDate' not in i.keys():
            player[i['id']] = year, i['id'], i['fullName'],i['firstName'], i['lastName'], i['birthDate'], i['birthCountry'], ft_to_m(i['height']), lb_to_kg(i['weight']), \
                                i['primaryPosition']['code'],i['primaryPosition']['name'], '1000-01-01', i['pitchHand']['code'], i['batSide']['code']
            player_affiliation[('{}_{}'.format(year, i['id']))] = i['currentTeam']['id']
            player_id[i['id']] = i['id']
        
        # 데이터 에러처리 (강정호 -> birthCountry : 한국)
        elif 'birthCountry' not in i.keys():
            player[i['id']] = year, i['id'], i['fullName'], i['firstName'], i['lastName'], i['birthDate'], 'Korea', ft_to_m(i['height']), lb_to_kg(i['weight']), \
                                i['primaryPosition']['code'],i['primaryPosition']['name'], i['mlbDebutDate'], i['pitchHand']['code'], i['batSide']['code']
            player_affiliation[('{}_{}'.format(year, i['id']))] = i['currentTeam']['id']
            player_id[i['id']] = i['id']

        else:
            player[i['id']] = year, i['id'], i['fullName'], i['firstName'], i['lastName'], i['birthDate'], i['birthCountry'], ft_to_m(i['height']), lb_to_kg(i['weight']), \
                                i['primaryPosition']['code'],i['primaryPosition']['name'], i['mlbDebutDate'], i['pitchHand']['code'], i['batSide']['code']
            player_affiliation[('{}_{}'.format(year, i['id']))] = i['currentTeam']['id']
            player_id[i['id']] = i['id']


In [ ]:
pd.DataFrame(player, index = ['season','player_id', 'full_name','first_name','last_name','birthDay','Country','height','weight','position_code','primary_position','mlbDebutDate','pitch_hand','batSide'])\
    .transpose().to_csv('./data/player.csv',index=False)

# player record

In [ ]:
# 데이터 저장
# for key, value in player_id.items():
#     if player[key][9] == '1':
#         pd.DataFrame(statsapi.player_stat_data(key, group="pitching", type="YearByYear")).to_json(f'./data/pitcher_record/p_{key}.json')
#     elif player[key][9] =='Y':
#         pd.DataFrame(statsapi.player_stat_data(key, group="[hittinng, pitching]", type="YearByYear")).to_json(f'./data/twin_record/t_{key}.json')        
#     else:
#         pd.DataFrame(statsapi.player_stat_data(key, group="hitting", type="YearByYear")).to_json(f'./data/batter_record/b_{key}.json')
    # time.sleep(0.1)

In [ ]:
pitchers_record = dict()
batters_record = dict()

for key, value in player_id.items():
    # 타자 / 투수 / twp entity를 분리하기 위한 조건문

    # 투수 스탯
    if player[key][9] == '1':
        p_record = pd.read_json(f'./data/pitcher_record/p_{key}.json')

        # 연도별 구분
        for record in p_record['stats']:
            id = '{}_{}'.format(record['season'], key)
            if id in player_affiliation.keys():
                
                pitchers_record[id] = p_record['id'][0], record['season'], player_affiliation[id], \
                    record['stats']['gamesPlayed'], record['stats']['homeRuns'], record['stats']['strikeOuts'], record['stats']['baseOnBalls'], record['stats']['era'], record['stats']['wins'], record['stats']['losses'], record['stats']['saves'], record['stats']['holds'], record['stats']['whip']

    # twp(오타니) 스탯
    elif player[key][9] =='Y':
        t_record = pd.read_json(f'./twin_record/t_{key}.json')

        # 연도별 구분
        for record in t_record['stats']:
            id = '{}_{}'.format(record['season'], key)
            if id in player_affiliation.keys():

                # 투수 / 타자 구분
                if record['group'] == 'pitching':
                    pitchers_record[id] = t_record['id'][0], record['season'], player_affiliation[id],\
                        record['stats']['gamesPlayed'], record['stats']['homeRuns'], record['stats']['strikeOuts'], record['stats']['baseOnBalls'], record['stats']['era'], record['stats']['wins'], record['stats']['losses'], record['stats']['saves'], record['stats']['holds'], record['stats']['whip']
                else:
                    batters_record[id] = t_record['id'][0],  record['season'], player_affiliation[id],\
                        record['stats']['gamesPlayed'], record['stats']['homeRuns'], record['stats']['runs'], record['stats']['strikeOuts'], record['stats']['baseOnBalls'], record['stats']['avg'], record['stats']['ops'], record['stats']['rbi'], record['stats']['stolenBases']
    
    # 타자 스탯
    else:
        b_record = pd.read_json(f'./batter_record/b_{key}.json')
         # 연도별 구분
        for record in b_record['stats']:
            id = '{}_{}'.format(record['season'], key)
            
            if id in player_affiliation.keys():
                batters_record[id] = b_record['id'][0], record['season'], player_affiliation[id],\
                    record['stats']['gamesPlayed'], record['stats']['homeRuns'], record['stats']['runs'], record['stats']['strikeOuts'], record['stats']['baseOnBalls'], record['stats']['avg'], record['stats']['ops'], record['stats']['rbi'], record['stats']['stolenBases']

        

In [ ]:
pd.DataFrame(pitchers_record,index = ['player_id','season', 'team_id', 'gamesPlayed', 'homeRuns', 'strikeOut', 'BB', 'era', 'win', 'loss', 'save', 'hold', 'whip']).transpose().to_csv('pitchers_record.csv',index=False)
pd.DataFrame(batters_record,index = ['player_id','season','team_id', 'gamesPlayed','homeRuns','runs','strikeOut','BB','avg','ops','rbi','steel']).transpose().to_csv('./data/batters_record.csv',index=False)

# coach affiliation

In [ ]:
# 데이터 저장
# for year in range(2013, 2024):
#     for key, value in team_id.items():
#         try:
#             pd.DataFrame(statsapi.get('team_coaches',{'teamId':value, 'season':year})).to_json(f'./data/coach_af/coach_{year}_{value}.json')
#         except:
#             print(f'{year}_{value}')


In [ ]:
# coach affiliation entity를 위한 10년 동안의 coach affiliation(type : dictionary) 생성 (attribute : year, person id, job, job id)
coach_affiliation = dict()

person_id = dict() 

for year in range(2013, 2024):
    for key, value in team_id.items():
        try:
            team_coach = pd.read_json(f'./data/coach_af/coach_{year}_{value}.json')

            for seq in range(len(team_coach['roster'])):
                
                attach = team_coach['roster'][seq]
                person_id[attach['person']['id']] = attach['person']['id']
                nameing = '{}_{}_{}'.format(year, value, attach['person']['id'])
                coach_affiliation[nameing] = year, value, attach['person']['id'], attach['job'], attach['jobId']
        except:
            continue


In [ ]:
pd.DataFrame(coach_affiliation,index = ['year','team_id','person_id','job','job_id']).transpose().to_csv('./data/coach_affiliation.csv',index=False)

# coach

In [ ]:
# for coach_id in person_id:
#     pd.DataFrame(statsapi.get('people',{'personIds':coach_id})['people'][0]).to_json(f'./coach/coach_{coach_id}.json')

In [ ]:
# coach entity를 위한 coach (type : dictionary) 
# 생일, 국적, 키, 몸무게의 데이터를 넣으려고 하였으나, 존재하지 않은 값들이 너무 많아 attribute를 id와 name으로 한정
coach = dict()

for coach_id in person_id:
    try:
        info = pd.read_json(f'./data/coach/coach_{coach_id}.json')
        coach[info['id'][0]] = info['id'][0], info['fullName'][0], info['firstName'][0], info['lastName'][0]
    except:
        continue

In [ ]:
pd.DataFrame(coach, index = ['person_id', 'full_name','first_name','last_name']).transpose().to_csv('./data/coach.csv',index=False)

# game

In [ ]:
# for key, value in team_id.items():
#     for ld in last_day:
#         year = ld[-4:]
#         pd.DataFrame(statsapi.schedule(start_date=f'01/01/{year}',end_date=ld,team=value)).to_json(f'./data/games/game_{value}_{year}.json')

In [ ]:
game_id = list() # 경기 상세 정보를 모으기 위한 게임 아이디 분류 리스트 -> 값의 중복이 없기때문에 list로 선언
game = dict() # 경기 ID, 경기 날짜, 홈 어웨이팀의 점수 데이터

for key, value in team_id.items():
    for year in range(2013, 2024):
        schedule = pd.read_json(f'./data/games/game_{value}_{year}.json')
        
        for i in range(len(schedule)):
            result = schedule.loc[i]

            if value == result['home_id']:
                game_id.append(result['game_id'])
                game[result['game_id']] = result['game_id'], result['game_date'], result['home_id'], result['home_score'], result['away_id'], result['away_score']

In [ ]:
pd.DataFrame(game, index = ['game_id', 'game_date','homeTeam','homeTeam_score','awayTeam','awayTeam_socre']).transpose().to_csv('./data/game.csv',index=False)

# game record

In [ ]:
# 데이터 저장
# pitcher = ['homePitchers', 'awayPitchers']
# batter = ['homeBatters', 'awayBatters']
# except_list = []

# for game_number in game_id:
#     try:
#         for rule in batter:
#             pd.DataFrame(statsapi.boxscore_data(game_number)[rule]).to_json(f'./data/{rule}/{game_number}.json')
        
#         for rule in pitcher:
#             pd.DataFrame(statsapi.boxscore_data(game_number)[rule]).to_json(f'./data/{rule}/{game_number}.json')

#         time.sleep(1)
        
#     except:
#         except_list.append(game_number)
#         time.sleep(10)
#         continue

In [ ]:
except_list

In [ ]:
pitcher = ['homePitchers', 'awayPitchers']
batter = ['homeBatters', 'awayBatters']

pitchers_game_record = dict()
batters_game_record = dict()

for game_number in game_id:
    for rule in batter:
        try:
            game_record = pd.read_json(f'./data/{rule}/{game_number}.json')
            for seq in range(1, len(game_record)): # 1 - last player number
                personId = game_record['personId'][seq]
                batters_game_record[f'{game_number}_{personId}'] = game_number, personId, game_record['ab'][seq], game_record['bb'][seq], game_record['h'][seq], game_record['hr'][seq], game_record['k'][seq], game_record['rbi'][seq], game_record['r'][seq], game_record['sb'][seq]
        except:
            continue
        
    for rule in pitcher:
        try:
            game_record = pd.read_json(f'./data/{rule}/{game_number}.json')
            for seq in range(1, len(game_record)):
                personId = game_record['personId'][seq]
                pitchers_game_record[f'{game_number}_{personId}'] = game_number, personId, game_record['ip'][seq], game_record['bb'][seq], game_record['k'][seq], game_record['h'][seq], game_record['hr'][seq]
        except:
            continue

In [ ]:
pd.DataFrame(pitchers_game_record, index = ['game_id', 'player_id','inning','BB','strikeOut','hits','homeRuns']).transpose().to_csv('./data/pitchers_game_record', index=False)
pd.DataFrame(batters_game_record, index = ['game_id', 'player_id','appearance','BB','hits','homeRun','strikeOut','rbi', 'r', 'stolen']).transpose().to_csv('./data/batters_game_record',index=False)